In [1]:
import pandas as pd 
import random
import numpy as np
import math
from sklearn.neighbors import NearestNeighbors

In [2]:
feature = ['Drama', 'Documentary', 'Family', 'Comedy', 'Crime',
               'Horror', 'Science Fiction', 'Action', 'Adventure',
               'War', 'Thriller', 'Romance', 'Fantasy', 'Mystery',
               'Western', 'Foreign', 'History', 'Music', 'Animation',
               'TV Movie', 'ab', 'af', 'am', 'ar', 'ay', 'bg', 'bm',
               'bn', 'bo', 'bs', 'ca', 'cn', 'cs', 'cy', 'da', 'de',
               'el', 'en', 'eo', 'es', 'et', 'eu', 'fa', 'fi', 'fr', 'fy',
               'gl', 'he', 'hi', 'hr', 'hu', 'hy', 'id_lang', 'is', 'it',
               'iu', 'ja', 'jv', 'ka', 'kk', 'kn', 'ko', 'ku', 'ky', 'la',
               'lb', 'lo', 'lt', 'lv', 'mk', 'ml', 'mn', 'mr', 'ms', 'mt',
               'nb', 'ne', 'nl', 'no', 'pa', 'pl', 'ps', 'pt', 'qu', 'ro',
               'ru', 'rw', 'sh', 'si', 'sk', 'sl', 'sm', 'sq', 'sr', 'sv',
               'ta', 'te', 'tg', 'th', 'tl', 'tr', 'uk', 'ur', 'uz', 'vi',
               'wo', 'xx', 'zh', 'zu', "vote_average",
               "revenue"] 

liked_movies = [['Heat',2], ['Ghost', 4]]

all_data = pd.read_csv("qwe.csv")

FileNotFoundError: File b'qwe.csv' does not exist

In [8]:
"""
Function to calculate the euclidean distance.
"""
def get_distance(data1, data2):
    #zipping the data allows you to easily compare column data between each data source
    points = zip(data1, data2)
    diffs_squared_distance = []
    for (a, b) in points:
        diffs_squared_distance.append(pow(a - b, 2))
    return math.sqrt(sum(diffs_squared_distance))

"""
Function to calculate similar movies based getting movie's with the 
shortest euclidean distance.
"""
def get_neighbor(movie_details, all_data, neighbor_num):
    distances = []
    
    #append the original title to the list of features so that we can recall which row correlates to each title
    feature_title = feature + ['original_title']

    #get the details of the whole row for  the movie which we are currently iterating on
    movie_row = all_data.loc[all_data['original_title'] == movie_details[0]]
    
    #get a filtered dataframe of just the movies that pertain to the cluster # of the movie we are looking at
    filtered_df = all_data.loc[all_data['clusters'] == movie_details[1]]

    #limit the dataframes to just the columns in the features list so that all the values are either ints or floats
    movie_row = movie_row[feature_title]
    filtered_df = filtered_df[feature_title]

    for index, row in filtered_df.iterrows():
        #compare the distances of each feature between the current liked movie and each row in the 
        dist = get_distance(movie_row.values[0][:-1], row[:-1])
        distances.append(dist)
 
    #routine to get the movie(s) that has the shortest distance
    
    count = 0
    new_rec_list = []
    
    #append to this remove_dist_list any time we go through the while loop
    #purpose is to avoid getting the same recommendation more than once 
    remove_dist_list = [0]
    
    #while count is less than the number of nearest neighbors we are requesting for
    while (count < neighbor_num):
        
        #get a distance that is greater than 0 because zero will just point to the same 'movie_row'
        mini_dist = min(x for x in distances if x not in remove_dist_list)
        
        remove_dist_list.append(mini_dist)
        
        #get the index of the selected shortest distance from the current movie
        mini_dist_index = distances.index(mini_dist)
        
        #get the name of the movie based on the index
        new_rec = filtered_df.iloc[mini_dist_index]['original_title']
        
        print("distance ::: " + str(mini_dist) + "   @ index ::: " + str(mini_dist_index) +"new rec ::::" + new_rec)

        new_rec_list.append(new_rec)
        
        #increment the count for the while statment
        count = count + 1
        
    return new_rec_list


"""
Function to iterate through all the liked movies that the front end will return
and then will get the nearest neighbors for those movies.
"""
def run_knn(liked_movies, all_data, neighbor_num):
    suggestion_list = []
    #iterate through liked moveis
    for movie in liked_movies:
        print(movie)
        
        #get nearest neighbors/recommendations
        new_rec = get_neighbor(movie, all_data, neighbor_num)
        suggestion_list.append(new_rec)
    
    #get all unique values
    flat_list = set([item for sublist in suggestion_list for item in sublist])
    return list(flat_list)

#neighbor number allows you to select how many nearest neighbors to return
neighbor_num = 2

#############EXAMPLE CALL
run_knn(liked_movies, all_data, neighbor_num)
        


['Heat', 2]
distance ::: 42700.0000941452   @ index ::: 274new rec ::::崖の上のポニョ
distance ::: 63182.000033633   @ index ::: 191new rec ::::Octopussy
['Ghost', 4]
distance ::: 949781.000001711   @ index ::: 6new rec ::::Aladdin
distance ::: 4811565.000000661   @ index ::: 136new rec ::::Rio 2


['崖の上のポニョ', 'Rio 2', 'Aladdin', 'Octopussy']